In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

log:
    root: '?'
    save_preds: '?' 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: '?'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

name: "OCTA_TempSweep"

experiment: 
    inf_kwargs:
        from_logits: True
        threshold: 0.5 
        temperature:
            # - (0.01, ..., 0.50, 0.025)
            # - (0.51, ..., 1.00, 0.025)
            # - (1.01, ..., 1.25, 0.025)
            # - (1.26, ..., 1.50, 0.025)
            - (1.51, ..., 2.00, 0.025)
            - (2.01, ..., 2.50, 0.025)
            - (2.51, ..., 2.75, 0.025)
            - (2.76, ..., 3.00, 0.025)
        pred_patch_dims: 
            height: 200 
            width: 200

# base_model: '/storage/vbutoi/scratch/ESE/training/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d'
base_model: '/storage/vbutoi/scratch/ESE/training/August_2024/08_07_24_OCTA_FULLRES_SoftDice/20240807_144440-P7H9-459da42e97888bccb0aa79c297f388cd'

model:
    checkpoint: 'min-val-abs_area_estimation_error'

inference_data:
    _class: "ese.datasets.OCTA_6M"
    split: ("cal", "val")

dataloader:
    batch_size: 1
    num_workers: 1

log:
    save_preds: False 

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(inf_cfgs)

4

## Running Jobs

In [8]:
from ese.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='3',
)

/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 40
Loading checkpoint: min-val-abs_area_estimation_error.
[Errno 2] No such file or directory: '/storage/vbutoi/scratch/ESE/training/August_2024/08_07_24_OCTA_FULLRES_SoftDice/20240807_144440-P7H9-459da42e97888bccb0aa79c297f388cd/checkpoints/min-val-abs_area_estimation_error.pt'
Defaulting to loading: max-val-dice_score.
Running:

dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  exp_name: OCTA_TempSweep
  exp_root: /storage/vbutoi/scratch/ESE/inference/10_08_24_OCTA_TempSweep
  inf_dataset_name: OCTA_6M
  inf_kwargs:
    pred_patch_dims:
      height: 200
      width: 200
    temperature: (1.51, 1.535, 1.5599999999999998, 1.5849999999999997, 1.6099999999999997,
      1.6349999999999996, 1.6599999999999995, 1.6849999999999994, 1.7099999999999993,
      1.7349999999999992, 1.7599999999999991, 1.784999999999999, 1.809999999999999,
      1.8349999999999989, 1.8599999999999988, 1.8849999999999987, 1.9099999999999986,
      1.9349999999999985, 1.9599999

  0%|          | 0/20 [00:00<?, ?it/s]


ValidationError: 1 validation error for ProcessPredMap
from_logits
  Field required [type=missing, input_value={'conf_map': tensor([[[[-....5, 'temperature': 1.51}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )